In [1]:
import datetime

import csv

import pandas as pd

from sqlalchemy import create_engine

import yfinance as yf 

from bs4 import BeautifulSoup

import requests

from pyspark.sql import SparkSession

import pyspark

from pyspark.context import SparkContext

import pyspark.sql.functions as fc

import pymysql

from pyspark.sql import Row

from pyspark.sql.types import *

import json

import numpy as np

In [2]:
sc = SparkContext.getOrCreate()

spark = SparkSession(sc)

### Connect to google cloud mysql 

In [6]:
connection = pymysql.connect(host='127.0.0.1',
                             user='root',
                             password='PApOEaPbJNOM62p1',
                             db='stockdb'
                            ,port=1234)
cur = connection.cursor()



#### From table CompanyInformation get distinct industry and store them into list Industry

In [7]:
sql1='SELECT Distinct Sector FROM CompanyInformation'
cur.execute(sql1)
Industry = cur.fetchall()

In [8]:
Industry

(('Health Care',),
 ('Industrials',),
 ('Consumer Discretionary',),
 ('Information Technology',),
 ('Consumer Staples',),
 ('Utilities',),
 ('Financials',),
 ('Real Estate',),
 ('Materials',),
 ('Energy',),
 ('Communication Services',))

## Get real-time price for s&p 500 companies
#### Function getprice returns a dictionary with Symbol as Key and price as Value

In [33]:
def getprice(name):
    #newdict=dict()
    newlist=[]
    for i in name:
        newdict=dict()
        url='https://finance.yahoo.com/quote/'
        finalurl=url+i+'?p='+i+'&.tsrc=fin-srch'
        r=requests.get(finalurl)
        soup = BeautifulSoup(r.content,'lxml')
        stockprice=soup.find_all('div',{"My(6px) Pos(r) smartphone_Mt(6px)"})[0].find('span').text
        newdict['name']=i
        newdict['price']=stockprice
        
        newlist.append(newdict)
    return newlist

### Get symbols of the company store them in list (companyname)

In [27]:
def industrycompany(cursor,Industry):
    sqlcode=('Select * From CompanyInformation Where Sector="'+Industry+'"')
    cursor.execute(sqlcode)
    Industry = cursor.fetchall()
    compname=[]
    for i in Industry:
        compname.append(i[0])
    #remove [:10] to return all names,  
    return compname[:10]

In [29]:
compname=industrycompany(cur,'Industrials')

#### Run getprice and return the dictionary contains real-time price

In [34]:
compdict=getprice(compname)

In [35]:
compdict

[{'name': 'AAL', 'price': '12.74'},
 {'name': 'ALK', 'price': '47.72'},
 {'name': 'ALLE', 'price': '114.56'},
 {'name': 'AME', 'price': '119.24'},
 {'name': 'AOS', 'price': '56.49'},
 {'name': 'BA', 'price': '203.30'},
 {'name': 'CARR', 'price': '39.39'},
 {'name': 'CAT', 'price': '172.74'},
 {'name': 'CHRW', 'price': '92.88'},
 {'name': 'CMI', 'price': '233.46'}]

### Spark RDD, and create spark dateframe to store the values in it 

In [36]:
rdd = sc.parallelize(compdict)
schema = StructType([ StructField("name", StringType(), True),StructField("price", StringType(), True)])

In [37]:
df3 = spark.createDataFrame(rdd, schema)

In [39]:
df3.show()

+----+------+
|name| price|
+----+------+
| AAL| 12.74|
| ALK| 47.72|
|ALLE|114.56|
| AME|119.24|
| AOS| 56.49|
|  BA|203.30|
|CARR| 39.39|
| CAT|172.74|
|CHRW| 92.88|
| CMI|233.46|
+----+------+



### Spark read 

In [40]:
stockspark = spark.read.json('stockdata.json')

companyspark = spark.read.json('companyinfo.json')

### Join two table 

In [41]:
sectorjoin=stockspark.join(companyspark,stockspark.name==companyspark.Symbol).select(stockspark.date,companyspark.Symbol,stockspark.open,stockspark.close,stockspark.high,stockspark.low,stockspark.volume,stockspark.dividends,companyspark.Sector)

### Didn't figure out how to use loop to pass in the industry name, so I am typing each industry in and create the spark df, below is example for energy spark table

In [42]:
n1=sectorjoin[(sectorjoin.Sector=='Energy')].groupBy(['Symbol','Sector']).agg(fc.max('close').alias('Period_High'),fc.min('close').alias('Period_Low'))

In [43]:
n2=n1.join(sectorjoin,sectorjoin.Symbol==n1.Symbol).filter(sectorjoin.close==n1.Period_High).select(sectorjoin.Sector,n1.Symbol,n1.Period_High,sectorjoin.date.alias('High_Date'),n1.Period_Low)

n3=n1.join(sectorjoin,sectorjoin.Symbol==n1.Symbol).filter(sectorjoin.close==n1.Period_Low).select(sectorjoin.Sector,n1.Symbol,n1.Period_High,n1.Period_Low,sectorjoin.date.alias('Low_Date'))

energytable=n2.join(n3,n3.Symbol==n2.Symbol).filter(n2.Period_High!= 'NaN').select(n2.Sector,n2.Symbol,n2.Period_High,n2.High_Date,n3.Period_Low,n3.Low_Date)

In [44]:
energytable.show(5)

+------+------+-----------+----------+----------+----------+
|Sector|Symbol|Period_High| High_Date|Period_Low|  Low_Date|
+------+------+-----------+----------+----------+----------+
|Energy|   OXY|       45.8|01/15/2020|      8.88|10/28/2020|
|Energy|   XOM|      65.66|01/02/2020|     29.54|03/23/2020|
|Energy|   OKE|      70.39|02/20/2020|     13.96|03/18/2020|
|Energy|   NOV|      25.36|01/06/2020|      8.01|10/28/2020|
|Energy|   EOG|      86.63|01/07/2020|     28.97|03/18/2020|
+------+------+-----------+----------+----------+----------+
only showing top 5 rows



### First method: try to upload spark into mysql by converting it into pandas first 

In [45]:
energydf = energytable.select("*").toPandas()

In [54]:
energydf.head()

,Sector,Symbol,Period_High,High_Date,Period_Low,Low_Date
0,Energy,OXY,45.80,01/15/2020,8.88,10/28/2020
1,Energy,XOM,65.66,01/02/2020,29.54,03/23/2020
2,Energy,OKE,70.39,02/20/2020,13.96,03/18/2020
3,Energy,NOV,25.36,01/06/2020,8.01,10/28/2020
4,Energy,EOG,86.63,01/07/2020,28.97,03/18/2020


### Use the code below to put spark into sql but the date data type is not working as expect, it convert date to a float decimal number

In [48]:
try:
    cur.execute("CREATE TABLE Industry_Energy (sector varchar(250),symbol varchar(250),periodhigh varchar(250),datehigh text,periodlow varchar(250),datelow text)")
    for row in energydf.iterrows():
        idx = row[0]
        sector = row[1][0]
        symbol = row[1][1]
        high = row[1][2]
        dateh=row[1][3]
        low=row[1][4]
        datel=row[1][5]
        cur.execute(f"INSERT INTO Industry_Energy (sector,symbol,periodhigh,datehigh,periodlow,datelow) VALUES ('{sector}','{symbol}', '{high}',{dateh},{low},{datel});")

except Exception as e:
    print(e)

In [52]:
sql_code=('Select * From Industry_Energy')
cur.execute(sql_code)
test1 = cur.fetchone()
test1

('Energy',
 'OXY',
 '45.8',
 '0.000033003300000000',
 '8.88',
 '0.000176803394554455')

### Second method 
### Export them into csv files for now, they are under code/datafiles

In [ ]:
energytable.coalesce(1).write.option("header","true").option("sep",",").mode("overwrite").csv("energy")

##  ....... 